## [타이타닉]

- 데이터 입력

In [1]:
import pandas as pd
import numpy as np


In [2]:
file = './titanic.csv'


In [3]:
titanicDF = pd.read_csv(file)


- 초기 데이터 확인

In [4]:
titanicDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


- 데이터 최적화

In [5]:
titanicDF = titanicDF.astype({
    'survived': 'bool',
    'sex': 'category',
    'embarked': 'category',
    'class': 'category',
    'who': 'category',
    'embark_town': 'category',
    'alive': 'category'
    })


In [6]:
titanicDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    bool    
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    category
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    category
 8   class        891 non-null    category
 9   who          891 non-null    category
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    object  
 12  embark_town  889 non-null    category
 13  alive        891 non-null    category
 14  alone        891 non-null    bool    
dtypes: bool(3), category(6), float64(2), int64(3), object(1)
memory usage: 50.5+ KB


- 결측치가 너무 많은 컬럼 제거

In [7]:
# 유효값이 700개 이하인 컬럼을 제거 : deck 컬럼을 제거함
titanicDF.dropna(thresh=700, axis='columns', inplace=True)


- 중복 컬럼 확인 및 제거

In [8]:
# survived 컬럼과 alive 컬럼 동일 여부 확인
(titanicDF.survived == (titanicDF.alive == 'yes')).value_counts()


True    891
Name: count, dtype: int64

In [9]:
# 두 컬럼이 동일하므로 alive 컬럼 제거
titanicDF.drop('alive', axis='columns', inplace=True)


In [10]:
titanicDF['class'].value_counts()


class
Third     491
First     216
Second    184
Name: count, dtype: int64

In [11]:
# pclass 컬럼과 class 컬럼 동일 여부 확인
# pclass 컬럼을 변환해보면 class 컬럼과 동일함
pcl = titanicDF['pclass'].replace({1:'First', 2:'Second', 3: 'Third'}).astype('category')
cl = titanicDF['class']
(pcl == cl).value_counts()


True    891
Name: count, dtype: int64

In [12]:
# 두 컬럼이 동일하므로 정보량이 더 적은 pclass 컬럼 제거
titanicDF.drop('pclass', axis='columns', inplace=True)


In [13]:
# embarked 컬럼과 embark_town 컬럼 동일 여부 확인
# embarked 컬럼은 embark_town 컬럼의 머릿글자임
emb1 = titanicDF[titanicDF['embarked'].notna()].embarked
emb2 = titanicDF[titanicDF['embark_town'].notna()].embark_town.str[0]
(emb1 == emb2).value_counts()


True    889
Name: count, dtype: int64

In [14]:
# 두 컬럼이 동일하므로 정보량이 더 적은 embarked 컬럼 제거
titanicDF.drop('embarked', axis='columns', inplace=True)


In [15]:
titanicDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    bool    
 1   sex          891 non-null    category
 2   age          714 non-null    float64 
 3   sibsp        891 non-null    int64   
 4   parch        891 non-null    int64   
 5   fare         891 non-null    float64 
 6   class        891 non-null    category
 7   who          891 non-null    category
 8   adult_male   891 non-null    bool    
 9   embark_town  889 non-null    category
 10  alone        891 non-null    bool    
dtypes: bool(3), category(4), float64(2), int64(2)
memory usage: 34.6 KB


- age 컬럼 결측치 대체작업

In [16]:
# 누락된 age 컬럼에 아이가 있는지 확인
# 모두 성인이므로 성인 나이의 평균치로 결측치 대체
(titanicDF['age'].isna() & (titanicDF['who'] == 'child')).value_counts()


False    891
Name: count, dtype: int64

In [17]:
# 성인 나이의 평균치를 정수로 추출
no_child = titanicDF['who'] != 'child'
adult_age_mean = titanicDF[no_child].mean(numeric_only=True)['age']
adult_age_mean = round(adult_age_mean)
adult_age_mean


33

In [18]:
# 최빈값 체크 결과 평균치와 동일함
titanicDF[no_child].mode(numeric_only=True)['age']


0    24.0
Name: age, dtype: float64

In [19]:
# age 컬럼의 결측치를 성인 나이의 평균치로 대체
titanicDF['age'] = titanicDF['age'].fillna(adult_age_mean).astype('int64')


In [20]:
titanicDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    bool    
 1   sex          891 non-null    category
 2   age          891 non-null    int64   
 3   sibsp        891 non-null    int64   
 4   parch        891 non-null    int64   
 5   fare         891 non-null    float64 
 6   class        891 non-null    category
 7   who          891 non-null    category
 8   adult_male   891 non-null    bool    
 9   embark_town  889 non-null    category
 10  alone        891 non-null    bool    
dtypes: bool(3), category(4), float64(1), int64(3)
memory usage: 34.6 KB


- embark_town 컬럼의 결측치 대체작업

In [21]:
# embark_town 컬럼은 탑승 항구를 의미하므로 결측치를 최빈값으로 대체
titanicDF['embark_town'] = titanicDF['embark_town'].fillna(titanicDF['embark_town'].mode()[0])


In [22]:
titanicDF.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    bool    
 1   sex          891 non-null    category
 2   age          891 non-null    int64   
 3   sibsp        891 non-null    int64   
 4   parch        891 non-null    int64   
 5   fare         891 non-null    float64 
 6   class        891 non-null    category
 7   who          891 non-null    category
 8   adult_male   891 non-null    bool    
 9   embark_town  891 non-null    category
 10  alone        891 non-null    bool    
dtypes: bool(3), category(4), float64(1), int64(3)
memory usage: 34.6 KB


- 생존률 분석 함수

In [23]:
def survival(column: str):
    if column == 'age':
        burkets = [titanicDF['age'].min(), 5, 12, 18, 29, 49, 64, titanicDF['age'].max()]
    else:
        burkets = None
    alive = titanicDF['survived']
    alive_col = titanicDF[alive][column].value_counts(bins=burkets)
    board = titanicDF[column].value_counts(bins=burkets)
    survive_rate = (alive_col / board).fillna(0).round(3).to_frame() * 100
    survive_rate.columns = ['survival rate(%)']
    if column == 'age':
        survive_rate.sort_index()
    return survive_rate


- 나이대별 생존 확률

In [40]:
# 영‧유아의 생존률이 유의미하게 높았고 나머지 나잇대는 큰 편차를 보이지 않음
suv_age = survival('age')
suv_age.index = ('영‧유아(0~5세)', '아동(6~12세)', '청소년(13~18세)', '청년(19~29세)',
                 '중년(30~49세)', '장년(50~64세)', '노년(65세 이상)')
suv_age


,survival rate(%)
영‧유아(0~5세),70.5
아동(6~12세),36.0
청소년(13~18세),42.9
청년(19~29세),35.1
중년(30~49세),36.7
장년(50~64세),41.3
노년(65세 이상),9.1


- 좌석별 생존 확률

In [41]:
# 좋은 좌석을 구매한 사람일수록 생존률이 높음
suv_class = survival('class')
suv_class.index = ('1등석', '2등석', '3등석')
suv_class


,survival rate(%)
1등석,63.0
2등석,47.3
3등석,24.2


- 남녀, 아이 생존률

In [42]:
# 여성의 생존률이 가장 높고, 아이는 절반, 남성이 가장 낮음
suv_who = survival('who')
suv_who.index = ('아이', '남성', '여성')
suv_who


,survival rate(%)
아이,59.0
남성,16.4
여성,75.6


- 남녀별 생존률

In [39]:
# 여성의 생존률이 높고 남성은 낮음
suv_gender = survival('adult_male')
suv_gender.index = ('여성', '남성')
suv_gender


,survival rate(%)
여성,71.8
남성,16.4


- 탑승 항구별 생존률

In [27]:
# 탑승 항구에 따른 생존률은 Cherbourg가 약간 높고 나머지 두 항구는 비슷함
survival('embark_town')


,survival rate(%)
embark_town,
Southampton,33.9
Cherbourg,55.4
Queenstown,39.0


- 동행자 유무별 생존률

In [46]:
# 동행자가 있을 때 생존률이 없을 때보다 약간 더 높음
suv_alone = survival('alone')
suv_alone.index = ('동행자 있음', '동행자 없음')
suv_alone


,survival rate(%)
동행자 있음,50.6
동행자 없음,30.4


- 함께 탑승한 형제자매 수별 생존률

In [29]:
# 형제자매의 수가 늘어나면 생존률이 감소하는 경향이 나타남
survival('sibsp')


,survival rate(%)
sibsp,
0,34.5
1,53.6
2,46.4
3,25.0
4,16.7
5,0.0
8,0.0


- 함께 탑승한 부모자식 수별 생존률

In [30]:
# 부모자식의 수가 4명 이상이면 생존률이 급감함
survival('parch')


,survival rate(%)
parch,
0,34.4
1,55.1
2,50.0
3,60.0
4,0.0
5,20.0
6,0.0
